<a target="_blank" href="https://colab.research.google.com/github/sakarimov/Basic-ML-Model/blob/main/JanKenPon%20CNN%20Image%20Recognizer.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Student Name** : Sulthan A. Karimov
# **Username** : sulthankarimov
# **Email** : sulthankarimov@gmail.com

In [1]:
!pip install split-folders[full] matplotlib jupyter_http_over_ws

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [3]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [4]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_COLAB

False

In [5]:
# sometime i run this notebook on my laptop for testing purpose
local_dir = (
    'datasets/images/' if IN_COLAB == False
    else '/tmp/'
)

dataset_name = 'rockpaperscissors'
local_data = local_dir + dataset_name
local_zip = local_data + '.zip'
print(local_zip)

datasets/images/rockpaperscissors.zip


In [6]:
!test -f $local_zip || wget --no-check-certificate \
 https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
 -o $local_zip

In [7]:
import zipfile, os, shutil, splitfolders, re

In [8]:
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(local_dir)
zip_ref.close()

In [9]:
shutil.rmtree(local_data + '/rps-cv-images')
os.remove(local_data + '/README_rpc-cv-images.txt')
print(os.listdir(local_data))

base_dir = local_dir + '/rps'
if os.path.exists(base_dir) == True:
  shutil.rmtree(base_dir)

splitfolders.ratio(local_data, base_dir, ratio=(.6,.4))
train_dir = os.path.join(base_dir, 'train')
print()
print('amount of training sample : ', sum(len(files) for _, _, files in os.walk(re.escape(base_dir) + r'/train')))
validation_dir = os.path.join(base_dir, 'val')
print('amount of validation sample : ', sum(len(files) for _, _, files in os.walk(re.escape(base_dir) + r'/val')))

['paper', 'rock', 'scissors']


Copying files: 2188 files [00:00, 5444.71 files/s]


amount of training sample :  1312
amount of validation sample :  876


In [10]:
os.listdir(train_dir)

['paper', 'rock', 'scissors']

In [11]:
os.listdir(validation_dir)

['paper', 'rock', 'scissors']

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    preprocessing_function = preprocess_input,
)

test_datagen = ImageDataGenerator(
    rescale = 1./255)

In [13]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'categorical',
    shuffle = False,
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'categorical',
    shuffle = False,
)

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [14]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths
import numpy as np
import pickle
import random
import os
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)
train_features = model.predict(train_generator)
train_features = train_features.reshape((train_features.shape[0], 7 * 7 * 512))
val_features = model.predict(validation_generator)
val_features = val_features.reshape((val_features.shape[0], 7 * 7 * 512))

[INFO] loading network...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718172438.489068     309 service.cc:145] XLA service 0x7869d00032a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718172438.489112     309 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 960M, Compute Capability 5.0


  2/328 ━━━━━━━━━━━━━━━━━━━━ 45s 141ms/step

I0000 00:00:1718172445.254437     309 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


328/328 ━━━━━━━━━━━━━━━━━━━━ 54s 144ms/step
219/219 ━━━━━━━━━━━━━━━━━━━━ 32s 146ms/step


In [15]:
train_features

array([[0.15979376, 0.        , 0.        , ..., 0.        , 0.6380874 ,
        0.        ],
       [0.14911175, 0.        , 0.        , ..., 0.00925446, 0.84793645,
        0.        ],
       [0.1327081 , 0.        , 0.        , ..., 0.1794551 , 0.6249763 ,
        0.        ],
       ...,
       [0.16579124, 0.        , 0.        , ..., 0.        , 0.5942712 ,
        0.        ],
       [0.09918394, 0.        , 0.        , ..., 0.        , 0.8006563 ,
        0.        ],
       [0.2933985 , 0.        , 0.        , ..., 0.        , 0.6294993 ,
        0.        ]], dtype=float32)

In [16]:
train_data = train_features.reshape(1312,7,14,256)
train_data.shape

(1312, 7, 14, 256)

In [17]:
# import the necessary packages
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import pickle
import os

trainX, trainY = train_features, train_generator.classes
testX, testY = val_features, validation_generator.classes

# train the model
print("[INFO] training model...")
modelLogReg = LogisticRegression(solver="lbfgs", multi_class="auto",
	max_iter=150)
modelLogReg.fit(trainX, trainY)
# evaluate the model
print("[INFO] evaluating...")
preds = modelLogReg.predict(testX)
print(classification_report(testY, preds, target_names=train_generator.class_indices.keys()))

[INFO] training model...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[INFO] evaluating...
              precision    recall  f1-score   support

       paper       0.98      0.99      0.99       285
        rock       0.98      1.00      0.99       291
    scissors       1.00      0.98      0.99       300

    accuracy                           0.99       876
   macro avg       0.99      0.99      0.99       876
weighted avg       0.99      0.99      0.99       876



In [18]:
import os
import numpy as np
#from google.colab import files
from tensorflow.keras.preprocessing import image
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

testdir = 'datasets/images/rps/test/'
uploaded = os.listdir(testdir)
fig = plt.figure(figsize= (10, 10))
#fig.tight_layout(pad=5.0)
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
for i in range(len(uploaded)):
    path = testdir + uploaded[i]
    img = image.load_img(path, target_size = (224,224))

    ax = fig.add_subplot(4, 4, i+1)
    ax.imshow(img)
    images = img_to_array(img)
    images = np.expand_dims(images, axis=0)
    images = preprocess_input(images)
    #images = np.vstack([x])
    features = model.predict(images, batch_size=4)
    features = features.reshape((features.shape[0], 7 * 7 * 512))
    pred = modelLogReg.predict(features)
    ax.title.set_text(list(train_generator.class_indices.keys())[int(pred[0])])


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/images/rps/test/'